# Inference

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch

import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist

pyro.set_rng_seed(101)

现在考虑一个简单的模型：

$$
weight|guess \sim Normal(guess, 1) \\
measurement|guess, weight \sim Normal(weight, 0.75)
$$

这是一个关于物体重量的模型，其随机性来自于两个方面：

1. 是这个物体本身的随机性，即这个物体可能是2kg，也可能是3kg，我们先验的认为是guess kg，则真实的weight是以此为均值的正态分布。
2. 是测量的随机性，即相同的物体在不同时刻测量有不同的measurement，这个也被认为是一个正态随机噪声。

In [2]:
# 使用pyro模型表示

def scale(guess):
    weight = pyro.sample("weight", dist.Normal(guess, 1.0))
    return pyro.sample("measurement", dist.Normal(weight, 0.75))
for _ in range(5):
    print(scale(10))

tensor(7.9981)
tensor(10.2329)
tensor(8.6991)
tensor(9.4201)
tensor(7.0255)


## Conditioning

现在对于我们进行推断的第一步，需要将观察的数据放入model中，即对其中的某个变量进行限制，然后才能进行推断。比如现在我们观察到我们的测量结果是9.5，然后我们希望去推断真实的weight是多少？

单纯限制其中某些变量、提供观察数据的工作由下面的几种方式来完成：

第一，使用`pyro.condition`函数，其接受一个model和观测，然后返回一个新的model（这个model接受的输入和之前的model一致）。注意，其只是做到了把观测放入模型，还没有求出后验分布。

In [3]:
def deferred_conditioned_scale(measurement, guess):
    return pyro.condition(scale, data={"measurement": measurement})(guess)
deferred_conditioned_scale(9.5, 10)

9.5

In [5]:
# 其返回的就是9.5，因为原始的model输出的是measurement的值，现在你固定了measurement的值，则其只会返回这个值。
deferred_conditioned_scale(9.5, 5)

9.5

第二，使用`pyro.sample`中的`obs`参数，其效果和`condition`一致

In [6]:
def scale_obs(guess):
    weight = pyro.sample("weight", dist.Normal(guess, 1.0))
    return pyro.sample("measurement", dist.Normal(weight, 0.75), obs=9.5)
scale_obs(10)

9.5

最后一种方式，是使用`pyro.do`，来自因果推断的语言。